In [1]:
from pylsl import StreamInfo, StreamOutlet
import numpy as np
import pandas as pd
from time import time, strftime, gmtime
from psychopy import visual, core, event
from glob import glob
from random import choice

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#name, type, channel_count, sampling rate, channel format, source_id
info = StreamInfo('Markers', 'Markers', 1, 0, 'int32', 'marker_id')

In [3]:
#make an outlet
outlet = StreamOutlet(info)

In [4]:
markernames = [1, 2]  #1 for Target, 2 for non-target; 0 is default for non-events

In [5]:
start = time()

In [6]:
#setup parameters
n_trials = 3
iti = .3
soa = 0.2
jitter = 0.2
record_duration = np.float32(120) #120.0

In [7]:
#setup appearance probability
position = np.random.binomial(1, 0.15, n_trials)  #randomize between 0 and 1, with 0.15 probability for 1, this is to set up the appearance of images

In [8]:
position

array([0, 1, 0])

In [9]:
trials = pd.DataFrame(dict(position=position, timestamp=np.zeros(n_trials)))

In [10]:
trials

,position,timestamp
0,0,0.0
1,1,0.0
2,0,0.0


In [11]:
def loadImage(filename):
    return visual.ImageStim(win=mywin, image=filename)

In [12]:
mywin = visual.Window([1920, 1080], fullscr=False)

targets = list(
        map(loadImage, glob('stim/cats_dogs/target-*.jpg')))  #map each file to loadImage(here) #cat is target
nontargets = list(
        map(loadImage, glob('stim/cats_dogs/nontarget-*.jpg')))

for ii, trial in trials.iterrows():
    # inter trial interval
    core.wait(iti + np.random.rand() * jitter)

    # onset
    pos = trials['position'].iloc[ii]  #running each position, using index as iterator (.iloc)
    image = choice(targets if pos == 1 else nontargets)  #if position == 1, randomly select images from targets
    image.draw()
    timestamp = time()
    outlet.push_sample([markernames[pos]], timestamp)  #(x, timestamp)
    mywin.flip()
    
    #offset
    core.wait(soa)
    mywin.flip()
    if len(event.getKeys()) > 0 or (time() - start) > record_duration:
        break
    event.clearEvents()
    
mywin.close()  #do not delete, otherwise, the window will not turn off

0.5948 	WARNING 	Monitor specification not found. Creating a temporary one...


In [13]:
mywin.close()